# Extraindo 20 mil músicas da base de 1 milhão

## Preparação

### Imports

Necessários o módulo pandas e nosso código custom para extrair as músicas

In [17]:
import pandas as pd
import requests  # para baixar a base/amostra para sua máquina
from tqdm.notebook import tqdm # para mostrar barra de progresso
import tarfile  # para descomprimir eventuais arquivos .tar.gz baixados
from libs.extraction_helpers import load_song_data


### Extensões

A extensão abaixo precisa ser ativada para que uma barra de progresso seja mostrada durante a operação.

In [10]:
!jupyter nbextension enable --py widgetsnbextension --sys-prefix

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK


### Localização do diretório que contém a base alvo (seja a base inteira ou o subset)


In [13]:
PATH_R = "../datasets/MillionSongSubset"

### Baixando o MillionSongSubset para simular a existência da base neste computador (opcional)

In [4]:
# Escrever o arquivo em disco, mostrando barra de progresso
# https://stackoverflow.com/a/37573701

url = "http://labrosa.ee.columbia.edu/~dpwe/tmp/millionsongsubset.tar.gz"
response = requests.get(url, stream=True)

# Levantar HTTPError pra qualquer resposta http não esperada
response.raise_for_status()

# Escrever e mostrar progresso
total_size_in_bytes= int(response.headers.get('content-length', 0))
block_size = 1024 # 1 Kb
progress_bar = tqdm(total=total_size_in_bytes, unit='iB', unit_scale=True)
output_filename = f"{PATH_R}.tar.gz"
with open(output_filename, 'wb') as file:
    for data in response.iter_content(block_size):
        progress_bar.update(len(data))
        file.write(data)
progress_bar.close()
if total_size_in_bytes != 0 and progress_bar.n != total_size_in_bytes:
    print("ERRO: algo deu errado!")

  0%|          | 0.00/1.98G [00:00<?, ?iB/s]

#### Descomprimir o arquivo baixado (opcional)

In [14]:
# open file
file = tarfile.open(output_filename)
  
# extracting file
file.extractall("../datasets/")
  
file.close()

In [15]:
# Localização de um arquivo de exemplo, para extrairmos os nomes de colunas etc
PATH_HDF5 = "../sample_data/TRAXLZU12903D05F94.h5"

In [18]:
df = load_song_data(dataset_root_dir=PATH_R, sample_hdf5_file_path=PATH_HDF5, letter="*", half=None, max_songs=100)

  0%|          | 0/10000 [00:00<?, ?it/s]

Informações básicas sobre o df:

In [19]:
df.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 54 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   analysis_sample_rate        100 non-null    int32  
 1   artist_7digitalid           100 non-null    int32  
 2   artist_familiarity          100 non-null    float64
 3   artist_hotttnesss           100 non-null    float64
 4   artist_id                   100 non-null    object 
 5   artist_latitude             100 non-null    float64
 6   artist_location             100 non-null    object 
 7   artist_longitude            100 non-null    float64
 8   artist_mbid                 100 non-null    object 
 9   artist_mbtags               100 non-null    object 
 10  artist_mbtags_count         100 non-null    object 
 11  artist_name                 100 non-null    object 
 12  artist_playmeid             100 non-null    int32  
 13  artist_terms                100 non-

In [20]:
df.describe()

,analysis_sample_rate,artist_7digitalid,artist_familiarity,artist_hotttnesss,artist_latitude,artist_longitude,artist_playmeid,danceability,duration,end_of_fade_in,...,mode,mode_confidence,release_7digitalid,song_hotttnesss,start_of_fade_out,tempo,time_signature,time_signature_confidence,track_7digitalid,year
count,100.0,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.0,100.000000,100.000000,...,100.000000,100.000000,100.00000,100.000000,100.000000,100.000000,100.000000,100.000000,1.000000e+02,100.000000
mean,22050.0,47875.680000,0.669029,0.465172,38.129662,-50.648945,21094.040000,0.0,231.362420,0.980030,...,0.660000,0.496220,289474.69000,0.461796,223.006950,128.336480,3.790000,0.573170,3.267322e+06,1995.540000
std,0.0,85154.874295,0.111780,0.106817,19.671481,51.327864,35001.766838,0.0,93.980866,1.879673,...,0.476095,0.193497,231530.91598,0.223831,92.137416,37.169436,1.174605,0.384708,2.539364e+06,10.934534
min,22050.0,158.000000,0.407925,0.267757,-36.354840,-123.207900,-1.000000,0.0,77.609340,0.000000,...,0.000000,0.000000,205.00000,0.000000,72.150000,66.410000,1.000000,0.000000,2.133000e+03,1963.000000
25%,22050.0,4225.750000,0.605931,0.402167,34.053490,-91.075710,-1.000000,0.0,165.374240,0.087000,...,0.000000,0.367500,79744.25000,0.334707,154.182250,97.371250,4.000000,0.155500,1.078504e+06,1990.000000
50%,22050.0,15189.500000,0.662950,0.444521,40.717390,-73.540450,6302.500000,0.0,216.489340,0.235000,...,1.000000,0.500000,279023.50000,0.501740,209.937000,123.424500,4.000000,0.684500,3.110612e+06,1998.000000
75%,22050.0,44305.500000,0.732276,0.505097,51.490705,-1.252925,20363.000000,0.0,276.642810,0.602000,...,1.000000,0.604500,398235.00000,0.630779,267.218250,150.993250,4.000000,0.954750,4.422651e+06,2004.000000
max,22050.0,466819.000000,0.934935,0.908203,57.153820,146.326110,192660.000000,0.0,600.371790,12.179000,...,1.000000,0.988000,812979.00000,0.984347,588.121000,237.466000,7.000000,1.000000,8.984739e+06,2010.000000


### Extraindo o dataframe pra arquivo

Se quisermos extrair o df como um json compresso (.json.gz):

In [21]:
df.to_json("../datasets/20kSongs.json.gz", orient="records", compression={'method': 'gzip', 'compresslevel': 9, 'mtime': 1})

Se quisermos extrair o df como arquivo pickle compresso (.pickle.gz):

In [16]:
df.to_pickle("../datasets/20kSongs.pickle.gz", compression={'method': 'gzip', 'compresslevel': 9, 'mtime': 1})

### Teste de carregamento do arquivo extraído

In [24]:
# Testar carregar o arquivo que acabamos de extrair, ainda compresso:
df_extracted = pd.read_pickle("../datasets/20kSongs.pickle.gz", compression={'method': 'gzip', 'compresslevel': 9, 'mtime': 1})

KeyboardInterrupt: 

In [25]:
df_extracted.info()

NameError: name 'df_extracted' is not defined